Raga vedere le patologie maggiori e diffuse tra gli obesi?

In [1]:
import pandas as pd
import numpy as np

# --- 1. Caricamento Dati ---
file_name = 'db_bmi.csv'
output_file = 'bmi_pathology_summary.csv' # Salviamo comunque il file completo

try:
    df = pd.read_csv(file_name)
    print(f"File '{file_name}' caricato con successo.")
except FileNotFoundError:
    print(f"ERRORE: File '{file_name}' non trovato.")
    exit()
except Exception as e:
    print(f"Errore nel caricamento: {e}")
    exit()

# --- 2. Identificazione Colonne ---
group_col = 'bmi_result'
cols_to_exclude = [
    'patient_id', 'sex', 'age', 'height', 'weight',
    'scp_codes', 'origine', 'bmi_value', 'bmi_result'
]
all_columns = df.columns.tolist()
pathology_columns = [col for col in all_columns if col not in cols_to_exclude]

if not pathology_columns:
    print("ERRORE: Non sono state trovate colonne di patologie.")
    exit()

print(f"Trovate {len(pathology_columns)} colonne di patologie.")


# --- 3. Filtro Dati ---
valid_categories = ['Sottopeso', 'Normopeso', 'Sovrappeso', 'Obeso']
df_filtered = df[df[group_col].isin(valid_categories)].copy()

if df_filtered.empty:
    print(f"ERRORE: Nessun paziente trovato nelle categorie: {valid_categories}")
    exit()

# --- 4. Raggruppamento e Conteggio (Come prima) ---
count_if_positive = lambda x: (x > 0).sum()
print("Raggruppamento e conteggio in corso...")
summary_df = df_filtered.groupby(group_col)[pathology_columns].agg(count_if_positive)
summary_df = summary_df.reindex(valid_categories)


# --- 5. Stampa delle Liste Ordinate ---
print("\n=======================================================")
print("--- Conteggio Patologie (> 0) per Categoria BMI ---")
print("   (Ordinate dalla più frequente alla meno frequente)")

# Iteriamo su ogni categoria (riga) del nostro sommario
for category in valid_categories:
    
    # Seleziona la riga (una Serie pandas) per questa categoria
    category_data = summary_df.loc[category]
    
    # Filtra: rimuovi le patologie con 0 conteggi
    category_data_filtered = category_data[category_data > 0]
    
    # Ordina i valori dal più alto al più basso
    sorted_data = category_data_filtered.sort_values(ascending=False)
    
    # ==========================================================
    # === INIZIO BLOCCO MODIFICATO ===
    # ==========================================================
    
    # Calcola il numero di patologie trovate per questo gruppo
    count_of_pathologies = len(sorted_data)
    
    print(f"\n\n--- {category.upper()} ---")
    print(f"Totale patologie trovate (> 0): {count_of_pathologies}") # <-- RIGA AGGIUNTA
    
    # ==========================================================
    # === FINE BLOCCO MODIFICATO ===
    # ==========================================================
    
    if sorted_data.empty:
        print("Nessuna patologia (conteggio > 0) trovata in questo gruppo.")
    else:
        # Stampa la Serie ordinata (che apparirà come una lista)
        print(sorted_data)

print("\n=======================================================")

# --- 6. Salvataggio (Opzionale, ma utile) ---
try:
    # Salva il riepilogo *completo* (non ordinato) in un file CSV
    summary_df.to_csv(output_file)
    print(f"\nRiepilogo completo salvato anche in: {output_file}")
except Exception as e:
    print(f"\nERRORE nel salvataggio del file: {e}")

File 'db_bmi.csv' caricato con successo.
Trovate 71 colonne di patologie.
Raggruppamento e conteggio in corso...

--- Conteggio Patologie (> 0) per Categoria BMI ---
   (Ordinate dalla più frequente alla meno frequente)


--- SOTTOPESO ---
Totale patologie trovate (> 0): 51
NORM       742
LVH        371
ASMI       322
LAFB       274
ISC_       221
NDT        214
IMI        175
IRBBB      149
PVC        117
IVCD        90
LAO/LAE     64
CRBBB       62
AMI         60
DIG         60
NST_        58
1AVB        57
CLBBB       54
ISCAL       49
RAO/RAE     47
ILMI        30
EL          29
LPFB        25
ISCIL       24
LMI         23
INJAS       21
ANEUR       21
PACE        19
ISCLA       19
ISCIN       17
ALMI        17
LNGQT       16
INJAL       16
ISCAS       15
ILBBB       13
RVH         12
SEHYP       11
IPLMI        6
PSVT         4
AFLT         3
INJLA        3
ISCAN        3
PAC          2
PMI          2
2AVB         2
BIGU         2
STACH        1
AFIB         1
IPMI         1
3AVB 

In [16]:
import pandas as pd

# --- 1. Carica il file ---
summary_df = pd.read_csv('bmi_pathology_summary.csv')

# --- 2. Imposta l'indice sulla colonna 'bmi_result' ---
summary_df = summary_df.set_index('bmi_result')

# --- 3. Categorie valide ---
valid_categories = ['Sottopeso', 'Normopeso', 'Sovrappeso', 'Obeso']

# --- 4. Creiamo un dizionario con le patologie presenti (>0) per ciascuna categoria ---
patologie_per_categoria = {}

for category in valid_categories:
    if category not in summary_df.index:
        print(f"ATTENZIONE: '{category}' non trovato nel file!")
        continue
    
    category_data = summary_df.loc[category]
    # Converte i valori in numerici (alcuni sono 'object' nel tuo CSV)
    category_data = pd.to_numeric(category_data, errors='coerce').fillna(0)
    patologie_presenti = set(category_data[category_data > 0].index)
    patologie_per_categoria[category.lower()] = patologie_presenti

# --- 5. Alias per comodità ---
sottopeso = patologie_per_categoria.get('sottopeso', set())
normopeso = patologie_per_categoria.get('normopeso', set())
sovrappeso = patologie_per_categoria.get('sovrappeso', set())
obeso = patologie_per_categoria.get('obeso', set())

# --- 6. Intersezioni principali ---
patologie_tutti_e_4 = sottopeso & normopeso & sovrappeso & obeso

patologie_sottopeso_normopeso = (sottopeso & normopeso) - (sovrappeso | obeso)
patologie_sottopeso_sovrappeso = (sottopeso & sovrappeso) - (normopeso | obeso)
patologie_sottopeso_obeso = (sottopeso & obeso) - (normopeso | sovrappeso)
patologie_normopeso_sovrappeso = (normopeso & sovrappeso) - (sottopeso | obeso)
patologie_normopeso_obeso = (normopeso & obeso) - (sottopeso | sovrappeso)
patologie_sovrappeso_obeso = (sovrappeso & obeso) - (sottopeso | normopeso)

# --- 7. Solo una categoria ---
patologie_solo_sottopeso = sottopeso - (normopeso | sovrappeso | obeso)
patologie_solo_normopeso = normopeso - (sottopeso | sovrappeso | obeso)
patologie_solo_sovrappeso = sovrappeso - (sottopeso | normopeso | obeso)
patologie_solo_obeso = obeso - (sottopeso | normopeso | sovrappeso)

# --- 8. Intersezioni triple ---
patologie_sottopeso_normopeso_sovrappeso = (sottopeso & normopeso & sovrappeso) - obeso
patologie_sottopeso_normopeso_obeso = (sottopeso & normopeso & obeso) - sovrappeso
patologie_sottopeso_sovrappeso_obeso = (sottopeso & sovrappeso & obeso) - normopeso
patologie_normopeso_sovrappeso_obeso = (normopeso & sovrappeso & obeso) - sottopeso

# --- 9. Stampa riepilogativa ---
print("\n=======================================================")
print("--- INTERSEZIONI E DIFFERENZE TRA CATEGORIE BMI ---")

def stampa_lista(nome, lista):
    print(f"\n{nome} ({len(lista)}):")
    if lista:
        print(sorted(lista))
    else:
        print("Nessuna patologia in questa combinazione.")

stampa_lista("patologie_tutti_e_4", patologie_tutti_e_4)
stampa_lista("patologie_solo_sottopeso", patologie_solo_sottopeso)
stampa_lista("patologie_solo_normopeso", patologie_solo_normopeso)
stampa_lista("patologie_solo_sovrappeso", patologie_solo_sovrappeso)
stampa_lista("patologie_solo_obeso", patologie_solo_obeso)
stampa_lista("patologie_sottopeso_normopeso", patologie_sottopeso_normopeso)
stampa_lista("patologie_normopeso_sovrappeso", patologie_normopeso_sovrappeso)
stampa_lista("patologie_sovrappeso_obeso", patologie_sovrappeso_obeso)
stampa_lista("patologie_sottopeso_obeso", patologie_sottopeso_obeso)
stampa_lista("patologie_sottopeso_normopeso_sovrappeso", patologie_sottopeso_normopeso_sovrappeso)
stampa_lista("patologie_sottopeso_normopeso_obeso", patologie_sottopeso_normopeso_obeso)
stampa_lista("patologie_sottopeso_sovrappeso_obeso", patologie_sottopeso_sovrappeso_obeso)
stampa_lista("patologie_normopeso_sovrappeso_obeso", patologie_normopeso_sovrappeso_obeso)

print("\n=======================================================")



--- INTERSEZIONI E DIFFERENZE TRA CATEGORIE BMI ---

patologie_tutti_e_4 (37):
['1AVB', 'ALMI', 'AMI', 'ANEUR', 'ASMI', 'BIGU', 'CLBBB', 'CRBBB', 'DIG', 'EL', 'ILBBB', 'ILMI', 'IMI', 'INJAL', 'INJAS', 'IPMI', 'IRBBB', 'ISCAL', 'ISCAS', 'ISCLA', 'ISC_', 'IVCD', 'LAFB', 'LAO/LAE', 'LMI', 'LNGQT', 'LPFB', 'LVH', 'NDT', 'NORM', 'NST_', 'PAC', 'PACE', 'PSVT', 'PVC', 'RAO/RAE', 'RVH']

patologie_solo_sottopeso (0):
Nessuna patologia in questa combinazione.

patologie_solo_normopeso (0):
Nessuna patologia in questa combinazione.

patologie_solo_sovrappeso (0):
Nessuna patologia in questa combinazione.

patologie_solo_obeso (0):
Nessuna patologia in questa combinazione.

patologie_sottopeso_normopeso (3):
['3AVB', 'AFIB', 'INJIN']

patologie_normopeso_sovrappeso (0):
Nessuna patologia in questa combinazione.

patologie_sovrappeso_obeso (0):
Nessuna patologia in questa combinazione.

patologie_sottopeso_obeso (0):
Nessuna patologia in questa combinazione.

patologie_sottopeso_normopeso_sovrapp

“Qual è la probabilità che un paziente sviluppi la patologia X dato che appartiene alla categoria Y?”

In [7]:
import pandas as pd

# === 1. Leggi il file CSV ===
df = pd.read_csv("bmi_pathology_summary.csv")

# === 2. Escludi le colonne con tutti 0 ===
# (mantieni solo quelle dove almeno un valore è diverso da zero)
df = df.loc[:, (df != 0).any(axis=0)]

# === 3. Calcola i totali per ogni categoria (somma riga) ===
df["Totale_categoria"] = df.drop(columns=["bmi_result"]).sum(axis=1)

# === 4. Calcola le probabilità condizionate ===
# Dividi ciascun valore per il totale della categoria
prob_df = df.copy()
for col in df.columns[1:-1]:  # escludi 'bmi_result' e 'Totale_categoria'
    prob_df[col] = df[col] / df["Totale_categoria"]

# === 5. Prepara la tabella finale (solo probabilità) ===
prob_df = prob_df.drop(columns=["Totale_categoria"])
prob_df = prob_df.set_index("bmi_result")

# === 6. (Opzionale) Trasponi per avere patologie come righe ===
prob_table = prob_df.T

# === 7. Visualizza le probabilità in formato leggibile (percentuali %) ===
print(prob_table.applymap(lambda x: f"{x*100:.2f}%"))

# Se vuoi salvare la tabella come CSV:
# prob_table.to_csv("probabilita_patologie_per_categoria.csv")


bmi_result Sottopeso Normopeso Sovrappeso   Obeso
NORM          20.92%    29.29%     49.51%  47.20%
IMI            4.93%     8.97%      7.01%   8.05%
AFLT           0.08%     0.12%      0.02%   0.00%
AFIB           0.03%     0.15%      0.00%   0.00%
NDT            6.03%     5.05%      8.06%   9.15%
LVH           10.46%     6.28%      5.12%   5.98%
LPFB           0.70%     0.56%      0.31%   0.12%
LAFB           7.72%     5.26%      2.99%   3.05%
IRBBB          4.20%     3.53%      3.73%   2.56%
RAO/RAE        1.33%     0.17%      0.07%   0.24%
RVH            0.34%     0.35%      0.57%   0.73%
IVCD           2.54%     2.56%      1.91%   2.20%
LMI            0.65%     0.67%      0.26%   0.12%
ASMI           9.08%     7.81%      2.78%   2.32%
AMI            1.69%     1.09%      0.86%   1.22%
ISCAL          1.38%     2.19%      1.24%   1.22%
1AVB           1.61%     2.69%      1.22%   1.22%
STACH          0.03%     0.00%      0.05%   0.00%
PACE           0.54%     0.93%      0.45%   0.37%


In [10]:
import pandas as pd

# === 1. Leggi il file ===
df = pd.read_csv("bmi_pathology_summary.csv")

# === 2. Rimuovi colonne con tutti 0 ===
df = df.loc[:, (df != 0).any(axis=0)]

# === 3. Totale pazienti per categoria (dato da te) ===
totali_categoria = {
    "Sottopeso": 2110,
    "Normopeso": 12610,
    "Sovrappeso": 3375,
    "Obeso": 656
}

# === 4. Calcola P(patologia | categoria) ===
prob_df = df.copy()
for idx, row in df.iterrows():
    categoria = row["bmi_result"]
    totale_cat = totali_categoria[categoria]
    for col in df.columns[1:]:
        prob_df.at[idx, col] = row[col] / totale_cat

prob_df = prob_df.set_index("bmi_result")

# === 5. Calcola P(patologia) complessiva su tutti i pazienti ===
total_pazienti = sum(totali_categoria.values())
p_global = {
    col: df[col].sum() / total_pazienti
    for col in df.columns[1:]
}

# === 6. Calcola Enrichment ratio ===
enrichment_df = prob_df.copy()
for col in enrichment_df.columns:
    enrichment_df[col] = enrichment_df[col] / p_global[col]

# === 7. Trasponi per avere le patologie come righe ===
enrichment_table = enrichment_df.T

# === 8. Mostra risultati arrotondati ===
print("\n=== ENRICHMENT RATIO ( >1 = prevalenza sopra la media ) ===\n")
print(enrichment_table.round(2))

enrichment_table.round(2).to_csv("patologie_pesate.csv")

# === 9. (Opzionale) Salva su CSV ===
# enrichment_table.to_csv("enrichment_ratio_by_category.csv")


/tmp/ipykernel_44/110448253.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.35165876777251187' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prob_df.at[idx, col] = row[col] / totale_cat
/tmp/ipykernel_44/110448253.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.08293838862559241' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prob_df.at[idx, col] = row[col] / totale_cat
/tmp/ipykernel_44/110448253.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.0014218009478672985' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prob_df.at[idx, col] = row[col] / totale_cat
/tmp/ipykernel_44/110448253.py:23: FutureWarning: Setting an item of incom

Interpretazione:

= 1 → la categoria ha lo stesso rischio medio della popolazione

> 1 → rischio superiore alla media

< 1 → rischio inferiore alla media

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=247b8f4a-5d0c-407e-bcc1-03b5acbab0dd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>